In [1]:
import os
import pandas as pd

In [2]:
li = []
for path, subdirs, files in os.walk(r'C:\Users\ivanr\Documents\berle-means\quantitative-methods\01_Lanitt\web_scraping_jobs\efg\catho_online'):
    for name in files:
        filename = os.path.join(path, name)
        df = pd.read_csv(filename, index_col=None, header=0, sep='|', encoding='UTF-16')
        df['palavra-chave'] = filename.split("\\")[-2]
        li.append(df)
df_raw = pd.concat(li, axis=0, ignore_index=True)

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5820 entries, 0 to 5819
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   titulo_vaga              5820 non-null   object
 1   salario_vaga             5820 non-null   object
 2   localidade_vaga          5820 non-null   object
 3   data_da_vaga             5820 non-null   object
 4   descricao_completa_vaga  5820 non-null   object
 5   palavra-chave            5820 non-null   object
dtypes: object(6)
memory usage: 272.9+ KB


In [4]:
df_raw['descricao_completa_vaga'][0]

'Criação de arte gráficas. Domínio em CorelDraw e atendimento ao público. Experiência na área de comunicação visual e gráfica.mostrar menoscompartilharQuero me candidatarBenefíciosRestaurante na empresa, Tíquete Alimentação, Vale TransporteHorárioDe segunda a sexta, das 08h à 18h.Regime de ContrataçãoCLT (Efetivo)Dados da EmpresaExclusivo para AssinantesUTILIZA A CATHO HÁ 1 ANO E 4 MESESQuero me candidatar'

In [5]:
description_list = df_raw['descricao_completa_vaga'].to_list()

In [6]:
description_list[1]

'Diretor de criação que saiba desenvolver layouts, supervisionar e criar rotina ´para a equipe de designer dentro da agência de publicidade. O profissional terá que colocar a mão na massa e executar também o trabalho além de supervisionar.Disponibilidade de horário para trabalho Que tenha conhecimento do pacote adobe Intenda de edição de vídeo e imagensmostrar menoscompartilharQuero me candidatarHorárioSegunda a sexta feira das 08:00hs as 18:00hsRegime de ContrataçãoPrestador de serviços (PJ)Informações AdicionaisQue saiba elaborar projetos de criaçãoDados da EmpresaExclusivo para AssinantesQuero me candidatar'

In [7]:
beneficios_list = []
logger = 0
for desc in description_list:
    try:
        beneficios = desc[desc.index("Benefícios")+10:desc.index("Horário")]
        beneficios_list.append(beneficios.strip().replace(",",";"))
    except:
        beneficios_list.append("Não Informado")

In [8]:
df_raw['beneficios'] = beneficios_list

In [9]:
description_list[1]

'Diretor de criação que saiba desenvolver layouts, supervisionar e criar rotina ´para a equipe de designer dentro da agência de publicidade. O profissional terá que colocar a mão na massa e executar também o trabalho além de supervisionar.Disponibilidade de horário para trabalho Que tenha conhecimento do pacote adobe Intenda de edição de vídeo e imagensmostrar menoscompartilharQuero me candidatarHorárioSegunda a sexta feira das 08:00hs as 18:00hsRegime de ContrataçãoPrestador de serviços (PJ)Informações AdicionaisQue saiba elaborar projetos de criaçãoDados da EmpresaExclusivo para AssinantesQuero me candidatar'

In [10]:
regime_de_contratacao_list = []
logger = 0
for desc in description_list:
    pj = ['pj','auto','autô','auton']
    clt = ['clt']
    temp = ['temporário','free']
    try:
        regime_de_contratacao = desc[desc.index("Regime de Contratação")+21:desc.index("Informações Adicionais")]
        if any(x in regime_de_contratacao.lower() for x in pj):
            regime_de_contratacao_list.append("Autônomo / PJ")
        elif any(x in regime_de_contratacao.lower() for x in clt):
            regime_de_contratacao_list.append("Efetivo / CLT")
        elif any(x in regime_de_contratacao.lower() for x in temp):
            regime_de_contratacao_list.append("Temporário")
        else:
            regime_de_contratacao_list.append(regime_de_contratacao.strip().replace(",",";"))
    except:
        regime_de_contratacao_list.append("Não Informado")

In [11]:
df_raw['regime_de_contratacao'] = regime_de_contratacao_list

In [12]:
description_list[1]

'Diretor de criação que saiba desenvolver layouts, supervisionar e criar rotina ´para a equipe de designer dentro da agência de publicidade. O profissional terá que colocar a mão na massa e executar também o trabalho além de supervisionar.Disponibilidade de horário para trabalho Que tenha conhecimento do pacote adobe Intenda de edição de vídeo e imagensmostrar menoscompartilharQuero me candidatarHorárioSegunda a sexta feira das 08:00hs as 18:00hsRegime de ContrataçãoPrestador de serviços (PJ)Informações AdicionaisQue saiba elaborar projetos de criaçãoDados da EmpresaExclusivo para AssinantesQuero me candidatar'

In [13]:
horario_list = []
logger = 0
for desc in description_list:
    padrao = ['segunda','sexta','8','18','seg','sex','2º a 6º','40']
    comercial = ['comercial','peri','integral',
                'das 07:30 h às 17:30 h. intervalo de 1h para almoço.',
                'das 09:00 às 19:00.',
                '7h30 ás 17h.',
                '07:30 a 17:00',
                'administrativo']
    flexivel = ['flexível', 'flexivel', 'livre','flex']
    combinar = ['combinar','sábado e feriados ocasionalmente.','disponibilidade para viagem.']
    flexivel_combinar = flexivel + combinar
    horas_44_semanais = ['44','hor']
    comercial_horas_44_semanais = comercial + horas_44_semanais
    shopping = ['shopp','shopping','shoping',
            '1 folga na semana 1 domingo de folga no mêss',
            'de domingo a domingo folga na semana.',
            'disponibilidade para trabalhar sábado e domingo com folga durante a semana.',
            'terça-feira a sábado']
    escala = ['escala','reveza','sistema 5x1.']
    escala_shopping = escala + shopping
    home_office = ['teletrabalho','home','off']
    plantao = ['00:00 ás 06:00','plantão','12 por 36.']
    meio_periodo = ['meio','parcial','07h30min às 11h30min / 13h30min às 17h30min',
                    '9h às 11h e 13h às 17h']
    vespertino = ['das 13:30 às 22:00.','16h às 22h.','13:30h as 22:00h',
                    '15:00 ás 23:20h','das 14hs às 22hs']
    nao_informado = ['indeterminada.','externo.',';mostrar menoscompartilharquero me candidatar','indefinido']
    try:
        horario = desc[desc.index("Horário")+7:desc.index("Regime de Contratação")]
        if any(x in horario.lower() for x in padrao):
            horario_list.append("Segunda a Sexta 8h as 18h")
        elif any(x in horario.lower() for x in comercial_horas_44_semanais):
            horario_list.append("Comercial")
        elif any(x in horario.lower() for x in flexivel_combinar):
            horario_list.append("A Combinar / Flexível")
        elif any(x in horario.lower() for x in escala_shopping):
            horario_list.append("Escala / Shopping")
        elif any(x in horario.lower() for x in home_office):
            horario_list.append("Home Office / Teletrabalho")
        elif any(x in horario.lower() for x in plantao):
            horario_list.append("Plantão")
        elif any(x in horario.lower() for x in meio_periodo):
            horario_list.append("Meio Período")
        elif any(x in horario.lower() for x in vespertino):
            horario_list.append("Vespertino")
        elif any(x in horario.lower() for x in nao_informado):
            horario_list.append("Não Informado")
        elif (not horario):
            horario_list.append("Não Informado")
        else:
            horario_list.append(horario.strip().replace(",",";"))
    except:
        horario_list.append("Não Informado")

In [14]:
df_raw['horario'] = horario_list

In [15]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5820 entries, 0 to 5819
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   titulo_vaga              5820 non-null   object
 1   salario_vaga             5820 non-null   object
 2   localidade_vaga          5820 non-null   object
 3   data_da_vaga             5820 non-null   object
 4   descricao_completa_vaga  5820 non-null   object
 5   palavra-chave            5820 non-null   object
 6   beneficios               5820 non-null   object
 7   regime_de_contratacao    5820 non-null   object
 8   horario                  5820 non-null   object
dtypes: object(9)
memory usage: 409.3+ KB


In [16]:
df_raw.to_excel(r'C:\Users\ivanr\Documents\berle-means\quantitative-methods\01_Lanitt\web_scraping_jobs\efg\parsed_data.xlsx', index=False)